In [ ]:
#  Code adapted from: https://github.com/aladdinpersson/Machine-Learning-Collection/blob/ac5dcd03a40a08a8af7e1a67ade37f28cf88db43/ML/Pytorch/GANs/2.%20DCGAN/train.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard.summary import hparams

import torchvision
import torchvision.transforms as tfms

import os, shutil
import math
import numpy as np
from itertools import product

Constants, etc.

In [ ]:
MAP_NAME = 'map_64x64'
MAP_DIMS = (64,64)
MAX_DATA_POINTS = 100
DATA_DIR = "./logs/"                     # name of directory that TensorBoard data is saved in

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

Set hyperparams for sweep

In [ ]:
parameters = dict(
    features_gen = [32, 64, 128],
    features_disc = [32, 64, 128],
    noise_channels = [32, 64],
    lr_gen = [1e-3, 1e-4, 1e-5],
    lr_disc = [1e-6, 1e-7, 1e-8],
    batch_size = [10, 50],
    num_epochs = [25]
)
param_values = [v for v in parameters.values()]
total_param_runs = np.prod([len(v) for v in parameters.values()])

Prepare data-processing functions

In [ ]:
transforms = tfms.Compose(
    [
        tfms.ToTensor(),
        tfms.Normalize([0.5 for _ in range(1)], [0.5 for _ in range(1)])
    ]
)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, features, device='cpu'):
        super(Discriminator, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, features, kernel_size=4, stride=2, padding=1, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(features, features * 2, kernel_size=4, stride=2, padding=1, bias=False, device=device),
            nn.BatchNorm2d(features * 2, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(features * 2, features * 4, kernel_size=4, stride=2, padding=1, bias=False, device=device),
            nn.BatchNorm2d(features * 4, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(features * 4, features * 8, kernel_size=4, stride=2, padding=1, bias=False, device=device),
            nn.BatchNorm2d(features * 8, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block5 = nn.Sequential(
            nn.Conv2d(features * 8, 1, kernel_size=4, stride=2, padding=0, device=device), # convert to single channel
            nn.AdaptiveAvgPool2d(1),    # pool the matrix into a single value for sigmoid
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.block1(x)
        y = self.block2(y)
        y = self.block3(y)
        y = self.block4(y)
        y = self.block5(y)

        return y

class Generator(nn.Module):
    def __init__(self, noise_channels, features, device='cpu'):
        super(Generator, self).__init__()

        self.block1 = nn.Sequential(
            nn.ConvTranspose2d(noise_channels, features * 16, kernel_size=4, stride=1, padding=0, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block2 = nn.Sequential(
            nn.ConvTranspose2d(features * 16, features * 8, kernel_size=4, stride=2, padding=1, bias=False, device=device),
            nn.BatchNorm2d(features * 8, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block3 = nn.Sequential(
            nn.ConvTranspose2d(features * 8, features * 4, kernel_size=4, stride=2, padding=1, bias=False, device=device),
            nn.BatchNorm2d(features * 4, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block4 = nn.Sequential(
            nn.ConvTranspose2d(features * 4, features * 2, kernel_size=4, stride=2, padding=1, bias=False, device=device),
            nn.BatchNorm2d(features * 2, device=device),
            nn.LeakyReLU(0.2)
        )

        self.block5 = nn.Sequential(
            nn.ConvTranspose2d(features * 2, 1, kernel_size=4, stride=2, padding=1, device=device),
            nn.Tanh()
        )

    def forward(self, x):
        y = self.block1(x)
        y = self.block2(y)
        y = self.block3(y)
        y = self.block4(y)
        y = self.block5(y)

        return y

In [ ]:
def noise_dims(map_dims):
    # For each convtranspose layer:
    # dim = round(math.ceil((dim + (2*padding) - kernel_size) / stride)) + 1
    # The output will need to be trimmed down to the desired size (This may hurt generalizability)
    shape = map_dims

    shape = round(math.ceil((shape + (2*1) - 4) / 2)) + 1  # Layer 5
    shape = round(math.ceil((shape + (2*1) - 4) / 2)) + 1  # Layer 4
    shape = round(math.ceil((shape + (2*1) - 4) / 2)) + 1  # Layer 3
    shape = round(math.ceil((shape + (2*1) - 4) / 2)) + 1  # Layer 2
    shape = round(math.ceil((shape + (2*0) - 4) / 2)) + 1  # Layer 1

    return shape

def trim(path, map_dims):
    resize = path.shape - map_dims

    axis_trim = (resize[0]//2, path.shape[0] - (resize[0]//2 + resize[0]%2))
    path = path[axis_trim[0]:axis_trim[1], :]

    axis_trim = (resize[1]//2, path.shape[1] - (resize[1]//2 + resize[1]%2))
    path = path[:, axis_trim[0]:axis_trim[1]]

    return path

In [ ]:
# redefining add_hparams() function in SummaryWriter class so that hyperparams are saved in a neater way
# code from https://github.com/pytorch/pytorch/issues/32651
class SummaryWriter(SummaryWriter):
    def add_hparams(self, hparam_dict, metric_dict):
        torch._C._log_api_usage_once("tensorboard.logging.add_hparams")
        if type(hparam_dict) is not dict or type(metric_dict) is not dict:
            raise TypeError('hparam_dict and metric_dict should be dictionary.')
        exp, ssi, sei = hparams(hparam_dict, metric_dict)

        logdir = self._get_file_writer().get_logdir()
        
        with SummaryWriter(log_dir=logdir) as w_hp:
            w_hp.file_writer.add_summary(exp)
            w_hp.file_writer.add_summary(ssi)
            w_hp.file_writer.add_summary(sei)
            for k, v in metric_dict.items():
                w_hp.add_scalar(k, v)

class PathsDataset(torch.utils.data.Dataset):
    def __init__(self, path, transform=None, shape = (100,100)):
        self.paths = [] # create a list to hold all paths read from file
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
                self.flat_path = np.loadtxt(f) # load in the flat path from file
                self.path = np.asarray(self.flat_path, dtype=int).reshape(len(self.flat_path)//2,2) #unflatten the path from the file

                self.path_matrix = self.convert_path(shape, self.path)
                
                self.paths.append(self.path_matrix) # add the path to paths list
        self.transform = transform
        print("Done!")

    def convert_path(self, map_dim, path):
        path_mat = np.zeros(map_dim, dtype=float)

        # Make the path continuous
        for i in range(path.shape[0] - 1):
            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            if (x1 < x2):
                x_dir = 1
            else:
                x_dir = -1

            if (y1 < y2):
                y_dir = 1
            else:
                y_dir = -1

            # Determine y from x
            if x2-x1 != 0:
                m = (y2-y1)/(x2-x1)
                while x != x2:
                    y = round(m*(x-x1) + y1)
                    path_mat[y,x] = 1
                    x += x_dir
            else:
                while x != x2:
                    path_mat[y1,x] = 1
                    x += x_dir


            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            # Determine x from y
            if y2-y1 != 0:
                m = (x2-x1)/(y2-y1)
                while y != y2:
                    x = round(m*(y-y1) + x1)
                    path_mat[y,x] = 1
                    y += y_dir
            else:
                while y != y2:
                    path_mat[y,x1] = 1
                    y += y_dir
            
        path_mat[path[path.shape[0]-1,1], path[path.shape[0]-1,0]] = 1     # Include the last point in the path

        return path_mat

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = np.float32(self.paths[idx])

        if self.transform:
            x = self.transform(x).cuda()
            

        return x

Prepare input

In [ ]:
dataset = PathsDataset(path=f'./env/{MAP_NAME}/paths/', shape=MAP_DIMS, transform=transforms)
# dataset = PathsDataset(path=f'./data/{MAP_NAME}/', shape=MAP_DIMS, transform=transforms)

NOISE_DIMS = noise_dims(MAP_DIMS)

In [ ]:
# reset data folder:
if os.path.isdir(DATA_DIR):
    shutil.rmtree(DATA_DIR)

In [ ]:
run_number = 0

for (features_gen, features_disc, noise_channels, lr_gen, lr_disc, batch_size, num_epochs) in product(*param_values):

    writer = SummaryWriter(f"{DATA_DIR}run{run_number}")

    disc = Discriminator(features_disc, device=DEVICE)
    gen = Generator(noise_channels, features_gen, device=DEVICE)
    fixed_noise = torch.randn((batch_size, noise_channels, NOISE_DIMS[0], NOISE_DIMS[1]), device=DEVICE)

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    opt_disc = optim.Adam(disc.parameters(), lr=lr_disc)
    opt_gen = optim.Adam(gen.parameters(), lr=lr_gen)
    criterion = nn.BCELoss()

    loss_step_rate = round((len(loader)*num_epochs)/MAX_DATA_POINTS)

    step = 0
    loss_step = 0
    img_step = 0

    for epoch in range(num_epochs):
        for batch_idx, real in enumerate(loader):
            batch_size = real.shape[0]

            ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
            noise = torch.randn((batch_size, noise_channels, NOISE_DIMS[0], NOISE_DIMS[1]), device=DEVICE)
            fake = gen(noise)
            disc_real = disc(real)
            lossD_real = criterion(disc_real, torch.ones_like(disc_real))
            disc_fake = disc(fake)
            lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
            lossD = (lossD_real + lossD_fake) / 2
            disc.zero_grad()
            lossD.backward(retain_graph=True)
            opt_disc.step()

            ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
            # where the second option of maximizing doesn't suffer from
            # saturating gradients
            output = disc(fake)
            lossG = criterion(output, torch.ones_like(output))
            gen.zero_grad()
            lossG.backward()
            opt_gen.step()

            if step % loss_step_rate == 0:              # save loss values at pre-determined rate
                # saving loss
                writer.add_scalar(f"Discriminator Loss", lossD, loss_step)
                writer.add_scalar(f"Generator Loss", lossG, loss_step)
                loss_step += 1

                print(
                    f"Run [{run_number+1}/{total_param_runs}] Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx}/{len(loader)}] \
                        Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
                )

            if batch_idx == len(loader)-1:              # save an image at the end of every epoch
                with torch.no_grad():
                    fake = gen(fixed_noise)
                    img_grid_fake = torchvision.utils.make_grid(
                        fake[:batch_size], normalize=True
                    )
                    img_grid_real = torchvision.utils.make_grid(
                        real[:batch_size], normalize=True
                    )

                    # saving output
                    writer.add_image(
                        f"Generator Output", img_grid_fake, global_step=img_step
                    )
                    writer.add_image(
                        f"Path Data", img_grid_real, global_step=img_step
                    )
                img_step += 1

            step += 1

    #saving hyperparams:
    writer.add_hparams({"gen_features": features_gen, "disc_features": features_disc, "noise_channels": noise_channels, "gen_lr": lr_gen, "disc_lr": lr_disc, "batch_size": batch_size, "epochs": num_epochs}, {"gen loss": lossG})
    writer.close()
    run_number += 1